In [1]:
import os
import sys
from itertools import permutations
import xlrd
import numpy as np
import pandas as pd

In [2]:
#计算两个向量长度
def len_v(vector1,vector2):
    return (np.linalg.norm(vector2),np.linalg.norm(vector1))

#计算两个向量的余弦相似度
def cos(vector1,vector2):
    vector1 = np.array(vector1)
    vector2 = np.array(vector2)
    return np.dot(vector1,vector2)/(np.linalg.norm(vector1)*np.linalg.norm(vector2))

#得到三个相似度
#a[2]是score
#a[2]两个向量长度组成的向量
def rate(a,b):
    sc1 = cos(a[0],b[0])
    sc2 = cos(a[1],b[1])
    sc3 = cos(a[3],b[3])
    return sc1,sc2,sc3

#三个相似度根据权重算出总的相似度
#theta = 0 只用长度
#theta = 1 
def conf(a,b, theta= 0.8):
    thre = 0.7
    if b>thre:
        return ((a[0]+a[1])*theta/2 +(1-theta)*a[2])
    else:
        return ((a[0]+a[1])*theta/2 +(1-theta)*a[2])*(-1)
    
    
def saveResult(res, savePath, videoName):
    if not os.path.exists(savePath):
          os.makedirs(savePath)
    fullPath = os.path.join(savePath, videoName + ".txt")
    print(fullPath)
    with open(fullPath,'w') as resultTxt:
        for i in range(len(res)):
            resultTxt.write(str(i)+" :"+str(res[i][1]))
            resultTxt.write("\n")

In [3]:
def getResult(dpFilePath, theta = 0.8):   
    videoPath = str(dpFilePath).rsplit("/",1)[0]
    videoName = videoPath.split("/")[-1]
    data = pd.read_csv(dpFilePath)
    x = data["X"].values
    y = data["Y"].values
    score = data["score"].values
    if len(x) < 3:
        print("Wrong Trajectory!")
        return None
    

    #从中间开始求相似度
    mid = int(len(x)/2)
    ub = []
    uf = []
    #向两边扩展
    for i in range(mid,0,-1):
        ub.append([(x[i]-x[i+1],y[i]-y[i+1]),(x[i-1]-x[i],y[i-1]-y[i]),score[i]])
    for i in range(mid,len(x)-1):
        uf.append([(x[i]-x[i-1],y[i]-y[i-1]),(x[i+1]-x[i],y[i+1]-y[i]),score[i+1]])
#     print("ub: {}\nuf: {}".format(ub, uf))

    #标准密码向量生成
    #0-8选三个点
    p3 = list(permutations([0,1,2,3,4,5,6,7,8],3))

    m = [(-1,1),(0,1),(1,1),
        (-1,0),(0,0),(1,0),
        (-1,-1),(0,-1),(1,-1)]

    #找任意三个点对应的向量组合
    vectors = []
    for p in p3:
        vectors.append([(m[p[1]][0]-m[p[0]][0],m[p[1]][1]-m[p[0]][1]),(m[p[2]][0]-m[p[1]][0],m[p[2]][1]-m[p[1]][1]),tuple([a+1 for a in p])])

    #把延伸的每三个点组成一个向量
    #加上长度向量
    for i in range(len(ub)):
        ub[i].append(len_v(ub[i][0],ub[i][1]))
    #加上长度向量
    for i in range(len(uf)):
        uf[i].append(len_v(uf[i][0],uf[i][1]))
    #标准向量加上长度向量
    for i in range(len(vectors)):
        vectors[i].append(len_v(vectors[i][0],vectors[i][1]))
    print(ub)
    #综合概率的阈值
    Threshold = 0.6

    #每三个点可能结果存起来
    rb = []
    for i in range(len(ub)):
        temp = []
        for v in vectors:
            ra = rate(ub[i],v)
            if (np.array(ra)>Threshold).all():
                temp.append([conf(ra,ub[i][2], theta),v[2]])
        temp.sort(reverse = True)
        rb.append(temp)
    rf = []
    for i in range(len(uf)):
        temp = []
        for v in vectors:
            ra = rate(uf[i],v)
            if (np.array(ra)>Threshold).all():
                temp.append([conf(ra,uf[i][2], theta),v[2]])
        temp.sort(reverse = True)
        rf.append(temp)

    # print("rb: {}".format(rb))    
    # print("rf: {}".format(rf))
    #拼接，最终结果    
    res = []
    start = (rb.pop(0),rf.pop(0))
    for b in start[0]:
        for f in start[1]:
            if f[1] == tuple(reversed(b[1])):
                res.append([f[0],list(f[1]),0,0])
    # print("res: {}".format(res))            
    #向两边延伸拼接
    num_r = 0
    while rf and rb:
        nb = rb.pop(0)
        nf = rf.pop(0)
        for b in nb:
            for r in res:
                if r[2] == num_r:
                    if b[1][0] == r[1][1] and b[1][1] == r[1][0] and b[1][2] not in r[1]: 
                        tu = r[1].copy()
                        tu.insert(0,b[1][2])
                        res.append([r[0]+b[0],tu,r[2]+1,r[3]])
        for f in nf:
            for r in res:
                if r[3] == num_r:
                    if f[1][0] == r[1][-2] and f[1][1] == r[1][-1] and f[1][2] not in r[1]:
                        tu = r[1].copy()
                        tu.append(f[1][2])
                        res.append([r[0]+f[0],tu,r[2],r[3]+1])
        num_r += 1

    if rb:
        nb = rb.pop(0)
        for b in nb:
            for r in res:
                if r[2] == num_r:
                    if b[1][0] == r[1][1] and b[1][1] == r[1][0] and b[1][2] not in r[1]:
                        tu = r[1].copy()
                        tu.insert(0,b[1][2])
                        res.append([r[0]+b[0],tu,r[2]+1,r[3]])
    #根据置信度排序结果
    res.sort(reverse = True)
    return res

In [4]:
def reconstruct(dpFilePath, savePath):
    videoPath = str(dpFilePath).rsplit("/",1)[0]
    videoName = videoPath.split("/")[-1]
    resDistance = np.array(getResult(dpFilePath, theta = 0))
    resAngle = np.array(getResult(dpFilePath, theta = 1))
    if(resAngle.any() == None or resDistance.any() == None):
        print("no result")
        return None
    i = 0
    res = []
    #删除小于4点的结果
    while i < (len(resAngle)):
        if(len(resAngle[i][1]) < 4):
            resAngle = np.delete(resAngle, i, axis = 0)
            continue
        i = i + 1
    while i < (len(resDistance)):
        if(len(resDistance[i][1]) < 4):
            resDistance = np.delete(resDistance, i, axis = 0)
            continue
        i = i + 1
    #在Distance 结果中查询Angle中的结果
#     resDistanceLen = len(resDistance) if len(resDistance) < 1000 else 1000
        
#     for i in range(resDistanceLen):
#         for j in range(len(resAngle)):
#             if (resDistance[i][1] == resAngle[j][1]):
#                 res.append([resDistance[i], j])
     #在Angle结果中查询Distance中的结果
    resAngleLen = len(resAngle) if len(resAngle) < 1000 else 1000
        
    for i in range(resAngleLen):
        for j in range(len(resDistance)):
            if (resAngle[i][1] == resDistance[j][1]):
                res.append([resAngle[i], j])
    res = np.array(res)
    if (len(res) != 0):
        res = res[np.argsort(res[:,-1])]
        res = res[:, 0]
        print(len(res))
    print(savePath)
    if not os.path.exists(savePath):
        os.makedirs(savePath)
        print("make dir: "+ savePath)
    
    saveResult(res, savePath, videoName)
    res_len = 20 if len(res) > 20 else len(res)
    for i in range(res_len):
        print(res[i])
    return (res)

In [6]:
# 打开标记成功的文件
# 1 成功， 0 失败
workbook = xlrd.open_workbook('experiment.xlsx')

# 查看工作表
print("sheets：" + str(workbook.sheet_names()))
# 通过文件名获得工作表,获取工作表1
table = workbook.sheet_by_name('遮挡')
myDict = {}
for i in range(1, len(table.col_values(0))):
    myDict[table.col_values(0)[i]] = table.col_values(8)[i]
# for keys,values in myDict.items():
#     print(keys, values)

sheets：['无遮挡', '遮挡', '开始画图案时间统计', 'DP算法实验']


In [7]:
resultPath = "results/hide_dis"
savePath = "/home/user/zy/attack-on-pattern-pin/results/hide_dis/hide_dis_results"
for roots, dirs, name in os.walk(resultPath):
    dirs.sort()
    for videoName in dirs:
        if(myDict.get(videoName) != 1):
            continue
#         if(videoName != "SSSM-B3-00088"):
#             continue
        print(videoName)
        noEndPath = os.path.join(resultPath, videoName, "opt_tra.csv")
        reconstruct(noEndPath, savePath)

SSSM-B3-00001
[[(-20, 2), (-5, -25), 1, (25.495097567963924, 20.09975124224178)], [(-5, -25), (-21, 21), 1, (29.698484809834994, 25.495097567963924)], [(-21, 21), (32, -23), 1, (39.408120990476064, 29.698484809834994)], [(32, -23), (4, -15), 1, (15.524174696260024, 39.408120990476064)], [(4, -15), (-33, 22), 1, (39.66106403010388, 15.524174696260024)], [(-33, 22), (14, 12), 1, (18.439088914585774, 39.66106403010388)]]
[[(-20, 2), (-5, -25), 1, (25.495097567963924, 20.09975124224178)], [(-5, -25), (-21, 21), 1, (29.698484809834994, 25.495097567963924)], [(-21, 21), (32, -23), 1, (39.408120990476064, 29.698484809834994)], [(32, -23), (4, -15), 1, (15.524174696260024, 39.408120990476064)], [(4, -15), (-33, 22), 1, (39.66106403010388, 15.524174696260024)], [(-33, 22), (14, 12), 1, (18.439088914585774, 39.66106403010388)]]
1000
/home/user/zy/attack-on-pattern-pin/results/hide_dis/hide_dis_results
make dir: /home/user/zy/attack-on-pattern-pin/results/hide_dis/hide_dis_results
/home/user/zy/a

1000
/home/user/zy/attack-on-pattern-pin/results/hide_dis/hide_dis_results
/home/user/zy/attack-on-pattern-pin/results/hide_dis/hide_dis_results/SSSM-B3-00006.txt
[6.597118322607114 list([4, 3, 1, 9, 6, 8, 7, 5, 2]) 5 1]
[6.429819555105066 list([1, 3, 4, 9, 6, 8, 7, 5, 2]) 5 1]
[6.26683468942564 list([4, 5, 1, 9, 3, 8, 7, 6, 2]) 5 1]
[6.068698213086488 list([4, 5, 1, 9, 2, 8, 7, 6, 3]) 5 1]
[6.4836456381315966 list([4, 6, 1, 9, 3, 8, 7, 5, 2]) 5 1]
[6.25721597951005 list([4, 2, 1, 9, 5, 8, 7, 6, 3]) 5 1]
[6.403836034272189 list([5, 3, 2, 9, 6, 8, 7, 4, 1]) 5 1]
[6.506824805899868 list([4, 3, 2, 9, 6, 8, 7, 5, 1]) 5 1]
[6.350873090270529 list([4, 3, 1, 9, 5, 8, 7, 6, 2]) 5 1]
[6.524794301935648 list([2, 3, 4, 9, 6, 8, 7, 5, 1]) 5 1]
[6.1835743227684805 list([1, 3, 4, 9, 5, 8, 7, 6, 2]) 5 1]
[6.246656831403956 list([4, 6, 2, 9, 3, 8, 7, 5, 1]) 5 1]
[5.672740408418895 list([4, 5, 1, 8, 3, 9, 7, 6, 2]) 5 1]
[6.142350423706172 list([5, 6, 2, 9, 3, 8, 7, 4, 1]) 5 1]
[6.314022318211177 list([

[[(-7, -2), (-10, -18), 1, (20.591260281974, 7.280109889280518)], [(-10, -18), (41, -27), 1, (49.09175083453431, 20.591260281974)], [(41, -27), (6, 15), 1, (16.15549442140351, 49.09175083453431)], [(6, 15), (-43, 15), 1, (45.5411901469428, 16.15549442140351)], [(-43, 15), (17, -13), 1, (21.400934559032695, 45.5411901469428)], [(17, -13), (-2, 15), 1, (15.132745950421556, 21.400934559032695)], [(-2, 15), (20, 0), 1, (20.0, 15.132745950421556)]]
1000
/home/user/zy/attack-on-pattern-pin/results/hide_dis/hide_dis_results
/home/user/zy/attack-on-pattern-pin/results/hide_dis/hide_dis_results/SSSM-B3-00010.txt
[6.398776179051406 list([3, 5, 8, 4, 6, 9, 7, 1, 2]) 6 0]
[6.112716465537716 list([3, 5, 8, 7, 6, 9, 4, 1, 2]) 6 0]
[6.530370811937226 list([5, 8, 4, 6, 9, 7, 1, 2, 3]) 5 1]
[6.244311098423537 list([5, 8, 7, 6, 9, 4, 1, 2, 3]) 5 1]
[6.121255667419239 list([1, 6, 9, 7, 4, 5, 3, 2, 8]) 3 3]
[6.338881658705357 list([2, 8, 1, 6, 9, 7, 4, 5, 3]) 5 1]
[6.211775686406727 list([8, 1, 6, 9, 7, 4

1000
/home/user/zy/attack-on-pattern-pin/results/hide_dis/hide_dis_results
/home/user/zy/attack-on-pattern-pin/results/hide_dis/hide_dis_results/SSSM-B3-00015.txt
[5.927791510794768 list([1, 6, 8, 4, 5, 3, 7, 2, 9]) 3 3]
[6.06679900925106 list([1, 6, 8, 7, 5, 3, 4, 2, 9]) 3 3]
[6.748556049422475 list([8, 6, 9, 7, 5, 3, 4, 1, 2]) 3 3]
[6.136094942114561 list([7, 2, 9, 5, 6, 3, 4, 1, 8]) 3 3]
[6.517340360150655 list([5, 3, 9, 7, 8, 6, 4, 1, 2]) 3 3]
[5.671777612527541 list([7, 2, 6, 4, 5, 3, 8, 1, 9]) 3 3]
[6.239130818476742 list([8, 3, 9, 4, 5, 2, 7, 1, 6]) 3 3]
[6.567113178820498 list([8, 3, 9, 7, 5, 2, 4, 1, 6]) 3 3]
[5.715427967419955 list([4, 2, 6, 7, 5, 3, 8, 1, 9]) 3 3]
[5.983701416586671 list([7, 2, 9, 4, 5, 3, 8, 1, 6]) 3 3]
[5.6283283600868055 list([1, 2, 6, 7, 5, 3, 8, 4, 9]) 3 3]
[6.374910377671797 list([7, 5, 9, 8, 6, 3, 4, 1, 2]) 3 3]
[6.22646004823163 list([4, 2, 9, 7, 5, 3, 8, 1, 6]) 3 3]
[6.142669320567882 list([4, 2, 9, 5, 6, 3, 7, 1, 8]) 3 3]
[6.5426719577749495 list([

[[(1, 10), (-9, 5), 1, (10.295630140987, 10.04987562112089)], [(-9, 5), (39, 13), 1, (41.10960958218893, 10.295630140987)], [(39, 13), (-19, 5), 1, (19.6468827043885, 41.10960958218893)], [(-19, 5), (-19, -30), 1, (35.510561809129406, 19.6468827043885)], [(-19, -30), (29, -1), 1, (29.017236257093817, 35.510561809129406)], [(29, -1), (4, 12), 1, (12.649110640673518, 29.017236257093817)], [(4, 12), (-32, 15), 1, (35.34119409414458, 12.649110640673518)], [(-32, 15), (12, -12), 1, (16.97056274847714, 35.34119409414458)]]
1000
/home/user/zy/attack-on-pattern-pin/results/hide_dis/hide_dis_results
/home/user/zy/attack-on-pattern-pin/results/hide_dis/hide_dis_results/SSSM-B3-00023.txt
[6.771540763460041 list([1, 6, 9, 7, 2, 3, 4, 5, 8]) 6 0]
[6.770530861099814 list([4, 6, 9, 7, 2, 3, 1, 5, 8]) 6 0]
[6.035443079938789 list([8, 7, 5, 6, 1, 2, 4, 3, 9]) 4 2]
[6.460573492671882 list([7, 5, 6, 1, 2, 8, 3, 9, 4]) 3 3]
[5.923962582022015 list([7, 5, 6, 1, 2, 9, 3, 8, 4]) 3 3]
[6.222359102091025 list(

...

[[(26, -3), (-26, -12), 1, (28.635642126552707, 26.1725046566048)], [(-26, -12), (3, -8), 1, (8.54400374531753, 28.635642126552707)], [(3, -8), (10, 1), 1, (10.04987562112089, 8.54400374531753)], [(10, 1), (11, -10), 1, (14.866068747318506, 10.04987562112089)], [(11, -10), (-3, -9), 1, (9.486832980505138, 14.866068747318506)], [(-3, -9), (8, 0), 1, (8.0, 9.486832980505138)], [(8, 0), (-5, 0), 1, (5.0, 8.0)]]
1000
/home/user/zy/attack-on-pattern-pin/results/hide_dis/hide_dis_results
/home/user/zy/attack-on-pattern-pin/results/hide_dis/hide_dis_results/SSSM-B4-00008.txt
[6.025377283765927 list([9, 8, 6, 2, 4, 1, 3, 7, 5]) 5 1]
[6.04166905779418 list([9, 8, 7, 4, 3, 5, 1, 6, 2]) 3 3]
[6.191618825784429 list([6, 8, 4, 3, 7, 1, 9, 5, 2]) 2 4]
[5.8254413051170015 list([6, 4, 1, 3, 5, 2, 9, 8, 7]) 2 4]
[6.065364677523236 list([9, 8, 6, 5, 4, 1, 3, 7, 2]) 5 1]
[6.134918135684825 list([8, 9, 7, 6, 2, 4, 1, 3, 5]) 6 0]
[5.569420708640838 list([3, 5, 1, 6, 4, 2, 9, 8, 7]) 2 4]
[6.00843044638496 l

[[(-4, -7), (-8, 24), 1, (25.298221281347036, 8.06225774829855)], [(-8, 24), (5, 5), 1, (7.0710678118654755, 25.298221281347036)], [(5, 5), (43, 3), 1, (43.104524124504614, 7.0710678118654755)], [(43, 3), (4, -15), 1, (15.524174696260024, 43.104524124504614)], [(4, -15), (-23, 19), 1, (29.832867780352597, 15.524174696260024)], [(-23, 19), (-25, 1), 1, (25.019992006393608, 29.832867780352597)], [(-25, 1), (28, -34), 1, (44.04543109109048, 25.019992006393608)], [(28, -34), (-1, -7), 1, (7.0710678118654755, 44.04543109109048)], [(-1, -7), (2, 4), 1, (4.47213595499958, 7.0710678118654755)]]
1000
/home/user/zy/attack-on-pattern-pin/results/hide_dis/hide_dis_results
/home/user/zy/attack-on-pattern-pin/results/hide_dis/hide_dis_results/SSSM-B4-00012.txt
[5.609663972097688 list([5, 3, 9, 6, 2, 1, 7, 4, 8]) 5 1]
[5.413451972363206 list([8, 5, 3, 9, 6, 2, 1, 7, 4]) 6 0]
[6.199361655100719 list([7, 4, 5, 9, 6, 2, 1, 8, 3]) 6 0]
[5.770980354774776 list([8, 4, 3, 9, 6, 2, 1, 7, 5]) 6 0]
[6.22102239

[[(4, -5), (7, -5), 1, (8.602325267042627, 6.4031242374328485)], [(7, -5), (12, 5), 1, (13.0, 8.602325267042627)], [(12, 5), (-22, 12), 1, (25.059928172283335, 13.0)], [(-22, 12), (13, -9), 1, (15.811388300841896, 25.059928172283335)], [(13, -9), (-11, -6), 1, (12.529964086141668, 15.811388300841896)]]
1000
/home/user/zy/attack-on-pattern-pin/results/hide_dis/hide_dis_results
/home/user/zy/attack-on-pattern-pin/results/hide_dis/hide_dis_results/SSSM-B4-00016.txt
[6.4028599448309 list([5, 1, 6, 8, 7, 4, 3, 2, 9]) 3 3]
[6.346418019398958 list([5, 1, 9, 8, 7, 4, 3, 2, 6]) 3 3]
[6.056068935898636 list([2, 3, 1, 6, 8, 7, 4, 9, 5]) 4 2]
[6.217733654413553 list([5, 3, 1, 9, 8, 7, 4, 6, 2]) 4 2]
[6.180500363012163 list([5, 6, 2, 9, 8, 7, 4, 3, 1]) 4 2]
[6.003143695320395 list([5, 3, 1, 6, 8, 7, 4, 9, 2]) 4 2]
[6.204126743610408 list([2, 3, 1, 9, 8, 7, 4, 6, 5]) 4 2]
[6.296426469675572 list([6, 1, 9, 8, 7, 4, 3, 2, 5]) 3 3]
[6.3250402463592055 list([5, 6, 1, 9, 8, 7, 4, 3, 2]) 4 2]
[6.334985206

[[(-33, 21), (24, -4), 1, (24.331050121192877, 39.11521443121589)], [(24, -4), (-12, -4), 1, (12.649110640673518, 24.331050121192877)], [(-12, -4), (11, -11), 1, (15.556349186104045, 12.649110640673518)], [(11, -11), (-21, 10), 1, (23.259406699226016, 15.556349186104045)], [(-21, 10), (10, -10), 1, (14.142135623730951, 23.259406699226016)]]
1000
/home/user/zy/attack-on-pattern-pin/results/hide_dis/hide_dis_results
/home/user/zy/attack-on-pattern-pin/results/hide_dis/hide_dis_results/SSSM-B4-00021.txt
[5.8725282923829845 list([6, 2, 9, 4, 5, 1, 3, 8, 7]) 4 2]
[5.868127220491473 list([5, 2, 9, 4, 6, 1, 3, 8, 7]) 4 2]
[6.375058956320679 list([7, 4, 6, 2, 3, 1, 5, 9, 8]) 4 2]
[5.397040611244501 list([5, 2, 6, 4, 9, 1, 3, 8, 7]) 4 2]
[5.93151034692192 list([4, 1, 9, 5, 6, 2, 3, 8, 7]) 4 2]
[5.759405386540451 list([9, 2, 6, 4, 5, 1, 3, 8, 7]) 4 2]
[5.998597990547244 list([6, 5, 9, 4, 2, 1, 3, 8, 7]) 4 2]
[6.161131420390355 list([7, 4, 5, 2, 3, 1, 6, 9, 8]) 4 2]
[6.556308163993192 list([9, 4,

[[(9, -4), (-9, 9), 1, (12.727922061357855, 9.848857801796104)], [(-9, 9), (21, -10), 1, (23.259406699226016, 12.727922061357855)], [(21, -10), (-21, -5), 1, (21.587033144922902, 23.259406699226016)], [(-21, -5), (9, -2), 1, (9.219544457292887, 21.587033144922902)]]
1000
/home/user/zy/attack-on-pattern-pin/results/hide_dis/hide_dis_results
/home/user/zy/attack-on-pattern-pin/results/hide_dis/hide_dis_results/SSSM-B4-00027.txt
[5.404608894456291 list([2, 1, 3, 4, 8, 7, 5, 9]) 3 2]
[5.361612205802279 list([9, 8, 3, 1, 5, 4, 2, 6]) 3 2]
[5.4859128243720345 list([2, 1, 6, 4, 8, 7, 5, 9]) 3 2]
[5.731765784534555 list([8, 7, 9, 1, 5, 4, 2, 6]) 3 2]
[5.69248194273562 list([8, 7, 6, 1, 5, 4, 2, 9]) 3 2]
[5.8287127358437 list([8, 7, 6, 1, 5, 4, 2, 3]) 3 2]
[5.4432864871057784 list([2, 1, 3, 4, 8, 7, 5, 6]) 3 2]
[5.571050659896415 list([8, 7, 3, 1, 5, 4, 2, 6]) 3 2]
[5.770443377184042 list([8, 7, 9, 1, 5, 4, 2, 3]) 3 2]
[5.160768269961944 list([8, 7, 9, 1, 3, 4, 2, 6]) 3 2]
[5.3320060617176015 l